# LAB | Introduction to MLOps

Answer the questions,

Data source: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
import pandas as pd

In [2]:
 #!pip install pyarrow

In [2]:
df_jan = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
df_feb = pd.read_parquet('./data/fhv_tripdata_2021-02.parquet')

In [3]:
df_jan.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [4]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1154112 entries, 0 to 1154111
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1154112 non-null  object        
 1   pickup_datetime         1154112 non-null  datetime64[us]
 2   dropOff_datetime        1154112 non-null  datetime64[us]
 3   PUlocationID            195845 non-null   float64       
 4   DOlocationID            991892 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1153227 non-null  object        
dtypes: datetime64[us](2), float64(2), object(3)
memory usage: 61.6+ MB


In [5]:
df_feb.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [6]:
df_feb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1037692 entries, 0 to 1037691
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1037692 non-null  object        
 1   pickup_datetime         1037692 non-null  datetime64[us]
 2   dropOff_datetime        1037692 non-null  datetime64[us]
 3   PUlocationID            153001 non-null   float64       
 4   DOlocationID            885340 non-null   float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1037692 non-null  object        
dtypes: datetime64[us](2), float64(2), object(3)
memory usage: 55.4+ MB


**Q1: Read the data for January. How many records are there?**

In [7]:
# your code here
record_count_jan = len(df_jan)
record_count_jan

1154112

In [8]:
# 1. Check the total number of standard missing (NaN) values
nan_count = df_jan['PUlocationID'].isna().sum()

print(f"Count of standard missing (NaN) values: {nan_count}")

Count of standard missing (NaN) values: 958267


In [9]:
# 2. Check the total number of -1 values
neg_one_count = (df_jan['PUlocationID'] == -1).sum()

print(f"Count of -1 sentinel values: {neg_one_count}")

Count of -1 sentinel values: 0


**Q2: What's the average trip duration in January?**

In [10]:
# your code here
df_jan['duration'] = df_jan['dropOff_datetime'] - df_jan['pickup_datetime']
print(df_jan['duration'])

0         0 days 00:17:00
1         0 days 00:17:00
2         0 days 01:50:00
3         0 days 00:08:17
4         0 days 00:15:13
                ...      
1154107   0 days 00:08:45
1154108   0 days 00:57:36
1154109   0 days 00:16:12
1154110   0 days 00:19:26
1154111   0 days 00:36:00
Name: duration, Length: 1154112, dtype: timedelta64[us]


In [11]:
#Convert to minutes
df_jan['duration'] = df_jan['duration'].apply(lambda x: x.total_seconds() / 60)
print(df_jan['duration'])

0           17.000000
1           17.000000
2          110.000000
3            8.283333
4           15.216667
              ...    
1154107      8.750000
1154108     57.600000
1154109     16.200000
1154110     19.433333
1154111     36.000000
Name: duration, Length: 1154112, dtype: float64


In [12]:
df_jan['duration'].describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [13]:
#Average trip duration
avg_duration_jan = df_jan['duration'].mean()
print(f'Average trip duration in January: {avg_duration_jan: .2f} minutes')

Average trip duration in January:  19.17 minutes


**Q3: How many records did you drop?**

In [14]:
# your code here
initial_count = len(df_jan)
print(f'Initial count: {initial_count}')

#Duration range 1 to 60 minutes
min_duration = 1
max_duration = 60

# Filter the data 
df_jan_filtered = df_jan[(df_jan['duration'] >= min_duration) & (df_jan['duration'] <= max_duration)].copy()

#Number of records dropped
records_dropped = initial_count - len(df_jan_filtered)
print(f'Number of records dropped : {records_dropped}')

# Update the df_jan to the filtered version for the rest of questions
df_jan = df_jan_filtered

Initial count: 1154112
Number of records dropped : 44286


**What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.**

In [15]:
# your code here
#Store the column names for location IDs
location_cols = ['PUlocationID' , 'DOlocationID']

#Fill NA values in locations ID columns with -1
df_jan[location_cols] = df_jan[location_cols].fillna(-1).astype('int64')

# Calculate the fraction of records where pickup location ID is -1
missing_pickup_fraction = (df_jan['PUlocationID'] == -1).mean()

print(f'The fraction of records with missing (now -1) PUlocationID is: {missing_pickup_fraction:.4f}')

The fraction of records with missing (now -1) PUlocationID is: 0.8353


**Q5: What's the dimensionality of this matrix? (The number of columns).**

In [16]:
# your code here
# One-hot encode PUlocationID and DOlocationID
X = pd.get_dummies(df_jan[['PUlocationID', 'DOlocationID']].astype(str))

# The dimensionality (number of columns) is:
dimensionality = X.shape[1]
print(f"The dimensionality of the one-hot encoded feature matrix is: {dimensionality}")


The dimensionality of the one-hot encoded feature matrix is: 525


**Q6: What's the RMSE on train?**

In [17]:
# your code here

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np

y = df_jan['duration']
ridge = Ridge()
ridge.fit(X, y)
y_pred = ridge.predict(X)
rmse_train = np.sqrt(mean_squared_error(y, y_pred))
print(f"The RMSE on the training data (January) is: {rmse_train:.2f}")

The RMSE on the training data (January) is: 10.53


Now, let's put data preprocssing steps in a function so that we can process the validation set in the same way as well.

In [18]:
# your code here
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
    """Applies necessary preprocessing steps to the dataframe."""

    # 1. Calculate and filter duration (1-60 mins)
    df['duration'] = df['dropOff_datetime'] - df['pickup_datetime']
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds() / 60)
    df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)].copy()

    # 2. Fill NA in Location IDs and convert to integer
    location_cols = ['PUlocationID', 'DOlocationID']
    df_filtered[location_cols] = df_filtered[location_cols].fillna(-1).astype('int64')

    return df_filtered

**Q7: What's the RMSE on validation?**

In [19]:
# your code here
#Preprocess February data
df_val = prepare_data(df_feb)

In [20]:
#  Create feature matrix for validation (X_val) 
X_val = pd.get_dummies(df_val[['PUlocationID', 'DOlocationID']].astype(str))
X_val = X_val.reindex(columns=X.columns, fill_value=0)
y_val = df_val['duration']

In [21]:
# Predict and calculate RMSE using the trained Ridge model
y_val_pred = ridge.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"The RMSE on the validation data (February) is: {rmse_val:.2f}")

The RMSE on the validation data (February) is: 11.01


## Why to use MLOps as we learn from this excercise ##

In [ ]:
# MLOps helps keep machine learning workflows consistent and reproducible, so results can be trusted and repeated.
# It enables monitoring and automation, making it easier to detect issues and retrain models when needed.
# By organizing code, data, and experiments, MLOps supports collaboration and reliable deployment in production.

## BONUS: 

Now, try and run this notebook on AWS Instance